In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

# import time
# # tmp sleep
# time.sleep(3600)
# print('sleep done')

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/lstm_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/nn_clean_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/cnn_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/nn_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_clean_1_feat.pkl
(95851, 6) (226998, 6)
file path

In [2]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [15]:
from sklearn.model_selection import KFold, StratifiedKFold
def simple_ens(model_name,k=3):
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for i in range(6):
        print(i,'column')
        kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
        val_loss_l,train_loss_l = 0,0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            # train for i class
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        print('this i avg train',i,train_loss_l,'avg val',val_loss_l)
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [16]:
xgb_res = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_csv_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0 column
0.0818378698914 0.0907616727899
0.0838293063169 0.0865344667011
0.0815418835668 0.0910910793019
this i avg train 0 0.082403019925 avg val 0.0894624062643
1 column
0.0179297024536 0.022652860574
0.0177039423658 0.0226864090403
0.0180781679445 0.0222391255769
this i avg train 1 0.0179039375879 avg val 0.0225261317304
2 column
0.0412733524467 0.0511565364571
0.0432141100576 0.0481376093153
0.0433362352291 0.0469474985977
this i avg train 2 0.0426078992445 avg val 0.0487472147901
3 column
0.00629005181968 0.00845909002941
0.00593366815579 0.00992458914217
0.00580088684871 0.0100491120148
this i avg train 3 0.00600820227473 avg val 0.00947759706212
4 column
0.0558558006868 0.0615262430836
0.0557857160691 0.0623121005582
0.056007398951 0.0615673415861
this i avg train 4 0.0558829719023 avg val 0.061801895076
5 column
0.0169533876009 0.0213503180376
0.0167932320374 0.0214025101364
0.016674893334 0.0216534700988
this i avg train 5 0.0168071709907 avg val 0.0214687660909
all train avg 